Model Training


In [148]:
import pandas as pd

In [149]:
df = pd.read_csv("gemstone.csv")

In [150]:
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [151]:
#finding null values
df.isnull().sum()

id         0
carat      0
cut        0
color      0
clarity    0
depth      0
table      0
x          0
y          0
z          0
price      0
dtype: int64

In [152]:
#removing the not needed column ie:ID
df = df.drop(labels=["id"],axis=1)

In [153]:
#removing the not needed column ie:ID
#Splitting the data for independent(X) and independent(Y) variable

X = df.drop(labels=["price"],axis=1)
Y = df[["price"]]

In [154]:
X.head()

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77


In [155]:
Y.head()

,price
0,13619
1,13387
2,2772
3,666
4,14453


In [156]:
# Define which columns should be ordinal-encoded and which should be scaled

categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [157]:
print(categorical_cols)
print(numerical_cols)

Index(['cut', 'color', 'clarity'], dtype='object')
Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')


In [158]:
for i in categorical_cols:
    print(X[i].unique())

['Premium' 'Very Good' 'Ideal' 'Good' 'Fair']
['F' 'J' 'G' 'E' 'D' 'H' 'I']
['VS2' 'SI2' 'VS1' 'SI1' 'IF' 'VVS2' 'VVS1' 'I1']


In [159]:
#define custom ranking for each ordinal variabls
#you have to have domain knowledge to get to know wich is lowest grade and which is highest
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [160]:
from sklearn.impute import SimpleImputer #to handle missing values
from sklearn.preprocessing import StandardScaler #feature scaling
from sklearn.preprocessing import OrdinalEncoder #ordinal encoding

#pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


In [161]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())

    ]

)

# Categorigal Pipeline
cat_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
    ('scaler',StandardScaler())
    ]

)

preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols),
('cat_pipeline',cat_pipeline,categorical_cols)
])


In [162]:
#Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.30,random_state=25)

In [163]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [164]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,1.603311,-0.576412,1.448246,1.513117,1.581904,1.488971,-0.132781,-0.318040,0.680311
1,0.303962,-0.299018,-0.116865,0.448527,0.492411,0.445151,0.872751,-0.318040,-1.314586
2,-0.887109,0.255771,-0.116865,-0.976942,-0.969326,-0.932110,0.872751,-0.934667,-0.649620
3,0.888669,-0.391483,-0.116865,0.998866,1.037158,0.967061,0.872751,0.298586,0.015346
4,0.910325,0.810560,-0.638569,1.016910,0.946367,1.068544,0.872751,1.531839,-0.649620


In [177]:
#model training 

from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [178]:
regression = LinearRegression()
regression.fit(X_train,Y_train)

LinearRegression()

In [179]:
regression.coef_

array([[ 6420.61836827,  -133.31478683,   -70.32041413, -1673.98565663,
         -526.22584022,   -80.17388825,    69.547439  ,  -461.7306698 ,
          648.1281239 ]])

In [180]:
regression.intercept_

array([3960.23923809])

In [209]:

def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae,mse, rmse, r2_square


In [212]:
models = {
    'linear_regression': LinearRegression(),
    "lasso": Lasso(),
    "ridge": Ridge(),
    "elasticNet": ElasticNet()
}

trained_model_list = []
model_list = []
r2_list = []

print(list(models))

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, Y_train)
    
    # Make prediction
    y_pred = model.predict(X_test)
    
    mae, mse, rmse, r2 = evaluate_model(Y_test, y_pred)  # Adjusted unpacking here
    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model Training Performance')
    print("RMSE:", rmse)
    print("MAE:", mae)
    print("MSE:", mse)  # Print MSE
    print("R2 score", r2 * 100)

    r2_list.append(r2)
    
    print('=' * 35)
    print('\n')

['linear_regression', 'lasso', 'ridge', 'elasticNet']
linear_regression
Model Training Performance
RMSE: 1014.9136406378979
MAE: 673.4234973219704
MSE: 1030049.6979528721
R2 score 93.74806057731074


lasso
Model Training Performance
RMSE: 1014.832086020644
MAE: 674.4532853634812
MSE: 1029884.1628170118
R2 score 93.74906530130006


ridge
Model Training Performance
RMSE: 1014.9163756446655
MAE: 673.4554253656584
MSE: 1030055.2495517039
R2 score 93.74802688159716


elasticNet
Model Training Performance
RMSE: 1546.9694430285901
MAE: 1068.4336287317067
MSE: 2393114.4576641866
R2 score 85.47486917319314


